<a href="https://colab.research.google.com/github/rahiakela/hands-on-machine-learning-with-scikit-learn-keras-and-tensorflow/blob/16-natural-nanguage-processing-with-RNNs-and-Attention/4_attention_mechanisms_transformer_architecture.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Attention Mechanisms: The Transformer Architecture

Consider the path from the word “milk” to its translation “lait” below. it is quite long! 

This means that a representation of this word (along with all the other words) needs to be carried over many steps before it is actually used. Can’t we make this path shorter?

<img src='https://github.com/rahiakela/img-repo/blob/master/hands-on-machine-learning-keras-tensorflow/simple-machine-translation-model.png?raw=1' width='800'/>

This was the core idea in a groundbreaking [2014 paper](https://arxiv.org/abs/1409.0473) by Dzmitry Bahdanau et al. They introduced a technique that allowed the decoder to focus on the appropriate words (as encoded by the encoder) at each time step.

For example, at the time step where the decoder needs to output the word “lait,” it will focus its attention on the word “milk.”

This means that the path from an input word to its translation is now much shorter, so the short-term memory limitations of RNNs have much less impact.

Attention mechanisms revolutionized neural machine translation and NLP in general), allowing a significant improvement in the state of the art, especially for long sentences (over 30 words).

<img src='https://github.com/rahiakela/img-repo/blob/master/hands-on-machine-learning-keras-tensorflow/encoder–decoder-attention-model.png?raw=1' width='800'/>

Instead of just sending the encoder’s final hidden state to the decoder (which is still done, although it is not shown in the figure), we now send all of its outputs to the decoder. 

At each time step, the decoder’s memory cell computes a weighted sum of all these encoder outputs: this determines which words it will focus on at this step. The weight α(t,i) is the weight of the ith encoder output at the tth decoder time step.

For example, if the weight α(3,2) is much larger than the weights α(3,0) and α(3,1), then the decoder will pay much more attention to word number 2 (“milk”) than to the other two words, at least at this time step. 

The rest of the decoder works just like earlier: at each time step the memory cell receives the inputs we just discussed, plus the hidden state from the previous time step, and finally (although it is not represented in the diagram) it receives the target word from the previous time step (or at inference time, the output from the previous time step).

But where do these α(t,i) weights come from? It’s actually pretty simple: they are generated by a type of small neural network called an alignment model (or an attention layer), which is trained jointly with the rest of the Encoder–Decoder model. This alignment model is illustrated on the righthand side.

This layer outputs a score (or energy) for each encoder output (e.g., e(3, 2)): this score measures how well each output is aligned with the decoder’s previous hidden state.

Finally, all the scores go through a softmax layer to get a final weight for each encoder output (e.g., α(3,2)). All the weights for a given decoder time step add up to 1 (since the
softmax layer is not time-distributed). 

This particular attention mechanism is called Bahdanau attention.
Since it concatenates the encoder output with the decoder’s previous hidden state, it is sometimes called concatenative attention (or additive attention).

## Setup

In [0]:
import sys
assert sys.version_info >= (3, 5)  # Python ≥3.5 is required

import sklearn 
assert sklearn.__version__ >= "0.20"  # Scikit-Learn ≥0.20 is required

# %tensorflow_version only exists in Colab.
try:
  %tensorflow_version 2.x
  IS_COLAB = True
except Exception:
  IS_COLAB = False
  pass

# TensorFlow ≥2.0 is required
import tensorflow as tf
from tensorflow import keras
assert tf.__version__ >= '2.0'

# Common imports
import numpy as np
import os

# to make this notebook's output stable across runs
np.random.seed(42)

# To plot pretty figures
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

if not tf.config.list_physical_devices('GPU'):
    print("No GPU was detected. LSTMs and CNNs can be very slow without a GPU.")
    if IS_COLAB:
        print("Go to Runtime > Change runtime and select a GPU hardware accelerator.")

## Visual Attention

Attention mechanisms are now used for a variety of purposes. One of their first applications beyond NMT was in generating image captions using [visual attention](https://arxiv.org/abs/1502.03044).

A convolutional neural network first processes the image and outputs some feature maps, then a decoder RNN equipped with an attention mechanism generates the caption, one word at a time. At each decoder time step (each word), the decoder uses the
attention model to focus on just the right part of the image.

For example, the model generated the caption “A woman is throwing a frisbee in a park,” and you can see what part of the input image the decoder focused its attention on when it was about to output the word “frisbee”: clearly, most of its attention was focused on the frisbee.

<img src='https://github.com/rahiakela/img-repo/blob/master/hands-on-machine-learning-keras-tensorflow/visual-attention.png?raw=1' width='800'/>

Attention mechanisms are so powerful that you can actually build state-of-the-art models using only attention mechanisms.

## Attention Is All You Need: The Transformer Architecture